In [1]:
import pandas as pd
from datasets import load_dataset

In [2]:
metadata_raw = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Video_Games", trust_remote_code=True)

In [3]:
metadata_raw_df = metadata_raw['full'].to_pandas()

In [4]:
metadata_raw_df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Video Games,Dash 8-300 Professional Add-On,5.0,1,[Features Dash 8-300 and 8-Q300 ('Q' rollout l...,[The Dash 8-300 Professional Add-On lets you p...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Aerosoft,"[Video Games, PC, Games]","{""Pricing"": ""The strikethrough price is the Li...",B000FH0MHO,None,None,None
1,Video Games,Phantasmagoria: A Puzzle of Flesh,4.1,18,[Windows 95],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Sierra,"[Video Games, PC, Games]","{""Best Sellers Rank"": {""Video Games"": 137612, ...",B00069EVOG,None,None,None
2,Video Games,NBA 2K17 - Early Tip Off Edition - PlayStation 4,4.3,223,[The #1 rated NBA video game simulation series...,[Following the record-breaking launch of NBA 2...,58.0,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['NBA 2K17 - Kobe: Haters vs Players...,2K,"[Video Games, PlayStation 4, Games]","{""Release date"": ""September 16, 2016"", ""Best S...",B00Z9TLVK0,None,None,None
3,Video Games,Nintendo Selects: The Legend of Zelda Ocarina ...,4.9,22,[Authentic Nintendo Selects: The Legend of Zel...,[],37.42,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon Renewed,"[Video Games, Legacy Systems, Nintendo Systems...","{""Best Sellers Rank"": {""Video Games"": 51019, ""...",B07SZJZV88,None,None,None
4,Video Games,Thrustmaster Elite Fitness Pack for Nintendo Wii,3.0,3,"[Includes (9) Total Accessories, Pedometer, Wi...",[The Thrustmaster Motion Plus Elite Fitness Pa...,None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",THRUSTMASTER,"[Video Games, Legacy Systems, Nintendo Systems...","{""Release date"": ""November 1, 2009"", ""Pricing""...",B002WH4ZJG,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137264,None,Story of Seasons: Pioneers Of Olive Town (Nint...,4.5,397,[A wild world of discovery - tame the wilderne...,"[Product Description, Inspired by Tales of you...",31.04,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Marvelous Europe,"[Video Games, Nintendo Switch, Games]","{""Release date"": ""March 26, 2021"", ""Best Selle...",B09XQJS4CZ,None,None,None
137265,Video Games,MotoGP 18 (PC DVD) UK IMPORT REGION FREE,4.0,1,[Brand new game engine - MotoGP18 has been reb...,[Become the champion of the 2018 MotoGP Season...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Milestone,"[Video Games, Game Genre of the Month]","{""Pricing"": ""The strikethrough price is the Li...",B07DGPTGNV,None,None,None
137266,Cell Phones & Accessories,Century Accessory Soft Silicone Protective Ski...,2.9,19,"[Easy access to all buttons, controls and port...",[This soft case cover will add a splash of col...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Century Accessory,"[Video Games, Legacy Systems, Xbox Systems, Xb...","{""Package Dimensions"": ""2.76 x 2.76 x 0.2 inch...",B00HUWCQBW,None,None,None
137267,None,Hasbro Interactive Mr. Potato Head Activity Pa...,3.9,5,[],"[Amazon.com, Everyone's favorite master-of-dis...",None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Hasbro,"[Video Games, PC, Games]","{""Release date"": ""July 24, 1999"", ""Best Seller...",B00002S9MH,None,None,None


In [5]:
with pd.option_context('display.max_colwidth', None):
    display(metadata_raw_df.sample(10))

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
2731,Video Games,MTV's Celebrity Deathmatch - PC,3.7,12,"[Play one of three different character match-ups in Story Mode, or choose from over 20 other characters in single match gaming, Real moves from the TV show and crazier moves you've never seen before, Original hosts Nick Diamond and Johnny Gomez, and referee Mills Lane]",[MTV's Celebrity Deathmatch gives you the chance to test your favorite - or least favorite - celebrities' strength in the wrestling ring!],5.0,"{'hi_res': [None, 'https://m.media-amazon.com/images/I/51WzbguWjjL._SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/21T7A865B9L.jpg', 'https://m.media-amazon.com/images/I/31BDm2VqflL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/21T7A865B9L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31BDm2VqflL._SX38_SY50_CR,0,0,38,50_.jpg'], 'variant': ['MAIN', 'GLMR']}","{'title': [], 'url': [], 'user_id': []}",2K,"[Video Games, PC, Games]","{""Release date"": ""October 14, 2003"", ""Best Sellers Rank"": {""Video Games"": 34897, ""PC-compatible Games"": 878}, ""Pricing"": ""The strikethrough price is the List Price. Savings represents a discount off the List Price."", ""Package Dimensions"": ""7.4 x 5.3 x 1.3 inches; 4.96 Ounces"", ""Type of item"": ""Video Game"", ""Rated"": ""Teen"", ""Item model number"": ""710425212161"", ""Is Discontinued By Manufacturer"": ""Yes"", ""Item Weight"": ""5 ounces"", ""Manufacturer"": ""2K"", ""Date First Available"": ""January 1, 2000""}",B0000CDHP2,None,None,None
104754,Video Games,PlayStation Classic (Renewed),4.2,228,"[Includes two wired Controllers, a virtual memory card and an HDMI cable]","[This pre-owned or refurbished product has been professionally inspected and tested to work and look like new. How a product becomes part of Amazon Renewed, your destination for pre-owned, refurbished products: A customer buys a new product and returns it or trades it in for a newer or different model. That product is inspected and tested to work and look like new by Amazon-qualified suppliers. Then, the product is sold as an Amazon Renewed product on Amazon. If not satisfied with the purchase, renewed products are eligible for replacement or refund under the Amazon Renewed Guarantee.]",84.89,"{'hi_res': ['https://m.media-amazon.com/images/I/71d7eJiLVzL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/61aiXIM0CbL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/710vnhljOWL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/61oJWvRYpiL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/611ZCJfgHIL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/61nyOFAgntL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/712fJ79rHyL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/61Cl8qk39zL._SL1000_.jpg', 'https://m.media-amazon.com/images/I/61KiMSa9SwL._SL1000_.jpg'], 'large': ['https://m.media-amazon.com/images/I/31g3vg8Ds4L.jpg', 'https://m.media-amazon.com/images/I/31MOumicruL.jpg', 'https://m.media-amazon.com/images/I/31pMkJNbrwL.jpg', 'https://m.media-amazon.com/images/I/21BAxJki6yL.jpg', 'https://m.media-amazon.com/images/I/21si9NAbSiL.jpg', 'https://m.media-amazon.com/images/I/31N2Zv7DACL.jpg', 'https://m.media-amazon.com/images/I/31BF578ll2L.jpg', 'https://m.media-amazon.com/images/I/51+asY8ibBL.jpg', 'https://m.media-amazon.com/images/I/515Isbu3ZeL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/31g3vg8Ds4L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31MOumicruL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31pMkJNbrwL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/21BAxJki6yL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/21si9NAbSiL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31N2Zv7DACL._SX38_SY50_CR,0,0,38,50_.jpg', '

# Build Feature Transformations

In [6]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [7]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline

In [8]:
# Custom function to reshape data from 1D to 2D
def reshape_2d_to_1d(X):
    return np.array(X).reshape(-1)

def flatten_string_array_col(X):
    return np.array(['\n'.join(x) for x in X])

def tfidf_pipeline_steps():
    steps = [
        ('impute', SimpleImputer(strategy='constant', fill_value='')),
        ('reshape', FunctionTransformer(reshape_2d_to_1d, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=500, ngram_range=(1, 2)))
    ]
    return steps

def description_pipeline_steps():
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', TfidfVectorizer(min_df=5, max_features=500, ngram_range=(1, 2)))
    ]
    return steps

def categories_pipeline_steps():
    def tokenizer(s):
        return s.split('\n')
    steps = [
        ('flatten_string_array_col', FunctionTransformer(flatten_string_array_col, validate=False)),
        ('tfidf', CountVectorizer(tokenizer=tokenizer, token_pattern=None))
    ]
    return steps

data = pd.Series(["from 14.99", "14.99", "price: 9.99", "20 dollars", "none"])
price_pattern = r'\b((?:\d+\.\d*)|(?:\d+))\b'
display(data.str.extract(price_pattern))

def price_parse_dtype(series, pattern):
    return series.str.extract(pattern).astype(float)

def price_pipeline_steps(price_pattern):
    steps = [
        ('extract_price', FunctionTransformer(price_parse_dtype, kw_args=dict(pattern=price_pattern), validate=False)),
        ('impute', SimpleImputer(strategy='constant', fill_value=0)),
        ('min_max_scale', MinMaxScaler())
    ]
    return steps

,0
0,14.99
1,14.99
2,9.99
3,20
4,NaN


In [9]:
tfm = [
    ('main_category', OneHotEncoder(handle_unknown='ignore'), ['main_category']),  # One-hot encoding for categorical field
    ('title', Pipeline(tfidf_pipeline_steps()), ['title']),  # TF-IDF vectorizer for text field
    ('description', Pipeline(description_pipeline_steps()), 'description'),  # TF-IDF vectorizer for another text field
    ('categories', Pipeline(categories_pipeline_steps()), 'categories'),  # Count Vectorizer for multi-label categorical
    ('price', Pipeline(price_pipeline_steps(price_pattern)), 'price')  # Normalizing price
]
cols = [x[0] for x in tfm]
cols

['main_category', 'title', 'description', 'categories', 'price']

In [10]:
train_full_df = pd.merge(train_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
val_full_df = pd.merge(val_df, metadata_raw_df[['parent_asin'] + cols], how='left', on='parent_asin')
train_full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
0,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B019WRM1IA,5.0,1451860309000,All Electronics,Microsoft Xbox 360 Wired Controller for Window...,"[Product Description, Precisely what you need ...","[Video Games, Legacy Systems, Xbox Systems, Xb...",67.83
1,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B08DHTZNNF,4.0,1538135312132,Computers,"Razer Mamba Elite Wired Gaming Mouse: 16,000 D...",[The Razer Mamba Elite features our acclaimed ...,"[Video Games, Legacy Systems, PlayStation Syst...",36.95
2,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B0C3MZ128V,5.0,1554755712398,Computers,Razer BlackWidow TE Chroma v2 TKL Tenkeyless M...,[The Razer BlackWidow Tournament Edition Chrom...,"[Video Games, PC, Accessories, Gaming Keyboards]",109.99
3,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B0BL3CW73P,4.0,1556830735236,Computers,"Logitech G332 Wired Gaming Headset, Rotating L...",[logitech G332 gaming headset lets you hear mo...,"[Video Games, PC, Accessories, Headsets]",29.99
4,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B07SBX48TY,4.0,1569958593706,Video Games,PowerA Spectra Enhanced Illuminated Wired Cont...,[Spectra has all the great features of our pop...,"[Video Games, Xbox One, Accessories, Controlle...",60.36
...,...,...,...,...,...,...,...,...,...
84814,AG7QNO4WAHSEKUSVQTFJ3LNKSYGQ,B001Q4HLFQ,5.0,1368386358000,Video Games,Guitar Hero Metallica - Playstation 3,"[Product description, Enter the world of Lars ...","[Video Games, Legacy Systems, PlayStation Syst...",51.0
84815,AG7QNO4WAHSEKUSVQTFJ3LNKSYGQ,B000OPPR2C,5.0,1368386442000,Video Games,Guitar Hero Encore: Rocks the 80's - PlayStati...,"[The 1980s were a crazy decade when big hair, ...","[Video Games, Legacy Systems, PlayStation Syst...",31.99
84816,AG7QNO4WAHSEKUSVQTFJ3LNKSYGQ,B00004YRQA,5.0,1368386539000,Computers,PlayStation 2 Memory Card (8MB),"[Product Description, PlayStation 2 Memory Car...","[Video Games, Legacy Systems, PlayStation Syst...",12.84
84817,AG7QNO4WAHSEKUSVQTFJ3LNKSYGQ,B001ELJE5Q,5.0,1368386631000,Video Games,Wii Guitar Hero World Tour Band Kit,"[Product description, We've all dreamed of bei...","[Video Games, Legacy Systems, Nintendo Systems...",None


In [11]:
train_full_df.to_parquet("../data/train_item_features.parquet")
val_full_df.to_parquet("../data/val_item_features.parquet")

# Transformation Pipeline